# Import Thienpont_2018/lambrechts data:

In [7]:
import scanpy as sc
import sys
sys.path.append("../../../scripts/")
import pandas as pd
import LCA_file_reading
import preprocessing
import reference_based_harmonizing

In [ ]:
project_dir =  "." # change to raw data dir HLCA core
target_adata = LCA_file_reading.read_file_Thienpont_2018(project_dir)

In [ ]:
n_genes_before = target_adata.shape[1]
sc.pp.filter_genes(target_adata, min_counts=1)
# drop annotation label that is automatically created:
target_adata.var.drop("n_counts", axis=1, inplace=True)
n_genes_after = target_adata.shape[1]
print("number of genes removed:", n_genes_before - n_genes_after)

In [ ]:
# add up genes with duplicate names
target_adata = preprocessing.add_up_duplicate_gene_name_columns(target_adata)
print(target_adata.shape, "\n")
# correct gene names
renamer_dict = preprocessing.get_gene_renamer_dict(target_adata.var.index.tolist().copy())
n_genes_to_rename = np.sum(old != new for old, new in renamer_dict.items())
print(n_genes_to_rename, "GENES TO BE RENAMED (old, new):\n")
for old, new in renamer_dict.items():
    if len(new) > 0:
        print(old, new)
target_adata.var["original_gene_names"] = target_adata.var.index.tolist().copy()
translation_dict = dict(zip(target_adata.var.index, target_adata.var.index))
for gene_to_rename, new_name in renamer_dict.items():
    translation_dict[gene_to_rename] = new_name
target_adata.var.index = target_adata.var.index.map(translation_dict)
# add up duplicate genes again
# they might have emerged due to gene re-naming above
target_adata = preprocessing.add_up_duplicate_gene_name_columns(target_adata)

In [ ]:
harmonizing_df = reference_based_harmonizing.load_harmonizing_table(
    "../../../supporting_files/metadata_harmonization/HLCA_anatomical_region_reference_mapping_20210521.csv"
)
consensus_df = reference_based_harmonizing.create_consensus_table(harmonizing_df)
celltype_translation_df = (
    reference_based_harmonizing.create_orig_ann_to_consensus_translation_df(
        target_adata, consensus_df, harmonizing_df, verbose=False
    )
)

In [ ]:
target_adata = reference_based_harmonizing.consensus_annotate_anndata(
    target_adata, celltype_translation_df, verbose=True
)

In [ ]:
target_adata.X[:30,:10].toarray()

#### store full raw object:

In [ ]:
target_adata.write("../../../data/HLCA_extended/extension_datasets/raw/Lambrechts/lambrechts.h5ad")

#### store subsetted object for scArches mapping (based on 2000 HLCA hvgs):

In [12]:
genes_to_keep = pd.read_csv("../query_datasets/genes_for_mapping.csv")

In [11]:
target_adata.var.head(5)

,original_gene_names
RP11-34P13.3,RP11-34P13.3
RP11-34P13.7,RP11-34P13.7
RP11-34P13.8,RP11-34P13.8
RP11-34P13.9,RP11-34P13.9
FO538757.3,FO538757.3


In [17]:
adata_subset = preprocessing.subset_and_pad_adata(genes_to_keep, target_adata)

not all genes were recovered, filling in 0 counts for 4 missing genes...


/home/icb/lisa.sikkema/miniconda3/envs/scRNAseq_analysis/lib/python3.7/site-packages/anndata/_core/merge.py:894: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  UserWarning,


In [18]:
adata_subset.write("../../../data/HLCA_extended/extension_datasets/ready/subsetted/lambrechts_sub.h5ad")

... storing 'original_gene_names' as categorical
... storing 'gene_symbols' as categorical
